In [1]:
import os
import pandas as pd
from tqdm import tqdm

data = []
for path, folder, files in os.walk('datasets/data/image'):
    if files:
        for f in files:
            data.append(((path + '/' + f), f[:-4]))

In [2]:
data = sorted(data, key=lambda x: x[1])

In [3]:
img_path, img_ids = list(zip(*data))

In [4]:
df_img = pd.DataFrame()
df_img['id'] = img_ids
df_img['img_path'] = img_path

In [5]:
labels = []
for path, folder, files in os.walk('datasets/data/label'):
    if files:
        for f in files:
            labels.append(((path + '/' + f), f[:-4]))

In [6]:
labels = sorted(labels, key=lambda x: x[1])
lbs_path, lbs_id = list(zip(*labels))

df_lbs = pd.DataFrame()
df_lbs['id'] = lbs_id
df_lbs['lbs_path'] = lbs_path

In [7]:
df = pd.merge(df_img, df_lbs, how='inner', on = 'id')
labels = df['lbs_path']

bbs = []
for path in tqdm(labels):
    with open(path) as f:
        content = f.readline().strip()  # Read the first line and remove leading/trailing whitespace
        first_value = content.split()[0]  # Split the line by whitespace and get the first element
    bbs.append(first_value)
    
df['label'] = bbs



100%|██████████| 136894/136894 [02:38<00:00, 863.48it/s] 


In [8]:
del df['lbs_path']
del df['id']

In [9]:
df

,img_path,label
0,datasets/data/image/77/160/2011/0002dbe4d037f0...,2
1,datasets/data/image/32/544/2012/0003114e4114a9...,2
2,datasets/data/image/84/1970/2012/0003748673d82...,4
3,datasets/data/image/149/1799/2011/0003a38d99bf...,1
4,datasets/data/image/157/1917/2011/0003d093bb65...,4
...,...,...
136889,datasets/data/image/77/164/2011/fffd4c0e312e69...,4
136890,datasets/data/image/100/248/2013/fffdf53f9e53f...,1
136891,datasets/data/image/118/957/2012/fffe1ecdeca31...,3
136892,datasets/data/image/122/603/2014/ffff702a098bb...,1


In [10]:
df['label'].value_counts()

4     49381
5     31174
3     23571
1     18455
2     13533
-1      780
Name: label, dtype: int64

In [11]:
indices_to_drop = df[df['label'] == '-1'].index

# Drop rows with the specified indices
df = df.drop(indices_to_drop)
df

,img_path,label
0,datasets/data/image/77/160/2011/0002dbe4d037f0...,2
1,datasets/data/image/32/544/2012/0003114e4114a9...,2
2,datasets/data/image/84/1970/2012/0003748673d82...,4
3,datasets/data/image/149/1799/2011/0003a38d99bf...,1
4,datasets/data/image/157/1917/2011/0003d093bb65...,4
...,...,...
136889,datasets/data/image/77/164/2011/fffd4c0e312e69...,4
136890,datasets/data/image/100/248/2013/fffdf53f9e53f...,1
136891,datasets/data/image/118/957/2012/fffe1ecdeca31...,3
136892,datasets/data/image/122/603/2014/ffff702a098bb...,1


In [12]:
print(df['label'].value_counts())

df['label'] = pd.to_numeric(df['label'], errors='coerce') - 1
df['label'].value_counts()


4    49381
5    31174
3    23571
1    18455
2    13533
Name: label, dtype: int64


3    49381
4    31174
2    23571
0    18455
1    13533
Name: label, dtype: int64

In [13]:
df.to_csv('compcars-angle.csv')